In [3]:
#A Simple Python template example
from string import Template
#Create a template that has placeholder for value of x
t = Template('x is $x')
#Substitute value of x in above template
print(t.substitute({'x' : 1}))

x is 1


In [4]:
#A Python Program to demonstrate the working of the string template
from string import Template

#List student that stores the name and marks
Student =[('Vishal',90),('Yashaswi',89),('Subhajit',94)]
#we are creating a basic structure to print the name and marks of the students.
t = Template('Hi $name, You have got $marks marks')

for i in Student:
    print(t.substitute(name=i[0], marks=i[1]))

Hi Vishal, You have got 90 marks
Hi Yashaswi, You have got 89 marks
Hi Subhajit, You have got 94 marks


In [5]:
from string import Template
template = Template('$name works as $role in $company')
string = template.safe_substitute(name='Subhajit',role='ML Engineer')
print(string)

Subhajit works as ML Engineer in $company


In [6]:
t = Template('I am $name from $city')
print('Template String = ', t.template)

Template String =  I am $name from $city


In [7]:
template = Template ('$$ is the symbol for $name')
string = template.substitute(name='Dollar')
print(string)

$ is the symbol for Dollar


In [8]:
template = Template('That $noun looks ${noun}y')
string = template.substitute(noun='Fish')
print(string)

That Fish looks Fishy


In [10]:
!pip install jinja2

In [12]:
from jinja2 import Template
t = Template("Hello {{ Something}}!")
t.render(Something="world")


'Hello world!'

In [13]:
t = Template("My favorite numbers: {% for n in range(1,10)%}{{n}}" "{%endfor%}")
t.render()

'My favorite numbers: 123456789'

In [16]:
!pip install jinjasql

In [17]:
user_transaction_template = '''
select 
    user_id
    , count(*) as num_transactions
    ,sum(amount) as total_amount
FROM
    transactions
where 
    user_id = {{user_id}}
    and transaction_date = {{ transaction_date}}
group BY    
    user_id
'''

In [22]:
params = {
    'user_id':1234,
    'transaction_date': '2019-03-02'
}

In [23]:
from jinjasql import JinjaSql
j = JinjaSql(param_style='pyformat')
query, bind_params = j.prepare_query(user_transaction_template,params)

In [24]:
print(query)


select 
    user_id
    , count(*) as num_transactions
    ,sum(amount) as total_amount
FROM
    transactions
where 
    user_id = %(user_id_1)s
    and transaction_date = %(transaction_date_2)s
group BY    
    user_id


In [25]:
print(bind_params)

{'user_id_1': 1234, 'transaction_date_2': '2019-03-02'}


In [26]:
print(query % bind_params)


select 
    user_id
    , count(*) as num_transactions
    ,sum(amount) as total_amount
FROM
    transactions
where 
    user_id = 1234
    and transaction_date = 2019-03-02
group BY    
    user_id


In [49]:
from six import string_types
isinstance(20190302, string_types)

False

In [44]:
def quote_sql_string(value):
    '''
    If value is a string type, escapes single quotes in the string
    and return the string enclosed in single quotes.
    '''
    if isinstance(value,string_types):
        new_value = str(value)
        new_value = new_value.replace("'","''")
        return "'{}'".format(new_value)
    return value

In [45]:
from copy import deepcopy

In [46]:
def get_sql_from_template(query, bind_params):
    if not bind_params:
        return query
    params = deepcopy(bind_params)
    for key, val in params.items():
        params[key] = quote_sql_string(val)
    return query % params

In [47]:
print(get_sql_from_template(query,bind_params))


select 
    user_id
    , count(*) as num_transactions
    ,sum(amount) as total_amount
FROM
    transactions
where 
    user_id = 1234
    and transaction_date = '2019-03-02'
group BY    
    user_id


In [ ]:
import sqlite3

conn = sqlite3.connect('test.db')
print("Opened Database Sucessfully")

In [61]:
conn.execute('''CREATE TABLE transactionsss
(transaction_id INT NOT NULL,
user_id int NOT NULL,
transaction_date date NOT NULL,
store_id int,
payment_method varchar(10),
amount float);''')

print("Table Created Sucessfully")

Table Created Sucessfully


In [62]:
conn.execute("INSERT INTO transactionsss (transaction_id,user_id,transaction_date,store_id, payment_method,amount)\
VALUES (1,1234,'2019-03-02',1,'Cash','500')");

conn.execute("INSERT INTO transactionsss (transaction_id,user_id,transaction_date,store_id, payment_method,amount)\
VALUES (1,1234,'2019-03-01',1,'Credit','300')");

conn.execute("INSERT INTO transactionsss (transaction_id,user_id,transaction_date,store_id, payment_method,amount)\
VALUES (1,1234,'2019-03-02',2,'Cash','200')");

conn.execute("INSERT INTO transactionsss (transaction_id,user_id,transaction_date,store_id, payment_method,amount)\
VALUES (1,1234,'2019-03-03',2,'Credit','100')");

conn.execute("INSERT INTO transactionsss (transaction_id,user_id,transaction_date,store_id, payment_method,amount)\
VALUES (1,4321,'2019-03-01',2,'Cash','500')");

conn.execute("INSERT INTO transactionsss (transaction_id,user_id,transaction_date,store_id, payment_method,amount)\
VALUES (1,4321,'2019-03-02',2,'debit','400')");

conn.execute("INSERT INTO transactionsss (transaction_id,user_id,transaction_date,store_id, payment_method,amount)\
VALUES (1,4321,'2019-03-03',1,'Cash','700')");

conn.commit()
print("Records Created Sucessfully")

Records Created Sucessfully


In [63]:
_BASIC_STATS_TEMPLATE = '''
select
    {{dim | sqlsafe}}
    , count(*) as num_transactions
    , sum(amount) as total_amount
    , avg(amount) as avg_amount
from
    transactionsss
group by
    {{dim | sqlsafe}}
order by total_amount desc
'''


In [65]:
def apply_sql_template(template,parameters, func_list=None):
    j = JinjaSql(param_style='pyformat')
    if func_list:
        for func in func_list:
            j.env.globals[func.__name__] = func
    query, bind_params = j.prepare_query(template,parameters)
    return get_sql_from_template(query, bind_params)

In [67]:
params = {
    'dim': 'payment_method'
}
sql = apply_sql_template(_BASIC_STATS_TEMPLATE, params)

In [71]:
def get_basic_stats_sql(dimensions):
    return apply_sql_template(_BASIC_STATS_TEMPLATE, params)

In [77]:
def get_basic_stats_sql(dimensions):
    params = {
        'dim': '\n  ,'.join(dimensions)
    }
    return apply_sql_template(_BASIC_STATS_TEMPLATE, params)

In [78]:
print(get_basic_stats_sql(['store_id','payment_method']))


select
    store_id
  ,payment_method
    , count(*) as num_transactions
    , sum(amount) as total_amount
    , avg(amount) as avg_amount
from
    transactionsss
group by
    store_id
  ,payment_method
order by total_amount desc
